In [ ]:
# spacy will be used for lemmatization and stop words removal it's better than nltk
!pip install spacy

In [ ]:
!pip install wordcloud

In [3]:
# Data analysis 
import pandas as pd
import numpy as np 

# Data plotting
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size' : 16})

import seaborn as sn
from wordcloud import WordCloud

# Text analysis
import re
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import emoji
import emot
import contractions
import string
import nltk
from nltk.stem import SnowballStemmer 
from nltk.tokenize import word_tokenize

# Modeling
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import naive_bayes
from sklearn.metrics import confusion_matrix, classification_report

# Save model
import pickle

import time

from keras import models, layers, optimizers
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras_preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pickle 
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense,LSTM,Dropout,Conv1D,MaxPooling1D,Flatten,BatchNormalization,Bidirectional,Lambda,GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

/home/verpen/anaconda3/lib/python3.9/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
2022-10-24 14:21:00.578779: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-24 14:21:00.578797: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-24 14:21:01.808533: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-10-24 14:21:01.808565: I tensorflow/stream_executor/cuda/cuda_d

# Loading data

In [2]:
# reviews = pd.read_csv('/home/verpen/Downloads/Datasets/Amazon_Reviews_4M/train.csv')
reviews = pd.read_csv('/home/verpen/Downloads/Datasets/Amazon_Reviews_4M/train_short_version.csv')

In [3]:
reviews

,Unnamed: 0,Review,Label
0,0,Good novel about gay youth: There are two book...,2
1,1,"Tudors Season I: This series is ""Awesome!"" It ...",2
2,2,Ah the memories.: What can be said about Rocko...,2
3,3,Cancer is not a Disese by Andreas Moritz: A ve...,2
4,4,better late than never: I more or less stumble...,2
...,...,...,...
19995,19995,2-Inch Visco Elastic Memory Foam Mattress Topp...,1
19996,19996,This doesn't deserve ONE star!: If you like al...,1
19997,19997,TERRIBLE!!!!: This story is HORRIBLE!!! It jus...,1
19998,19998,"Half a star actually, if that..: As an artist ...",1


# Data analysis


### Analysis of the dataset

In [4]:
# Check if null values exist
reviews.isnull().value_counts()

Unnamed: 0  Review  Label
False       False   False    20000
dtype: int64

In [5]:
# Number of positive and negative reviews
reviews['Label'].value_counts()

2    10000
1    10000
Name: Label, dtype: int64

In [6]:
# Choose a random subset from the dataset with equal number of 
# positive and negative tweets
# subset = 10000
# pos_subset = reviews[reviews['Label'] == 2].sample(subset, random_state = 43)
# neg_subset = reviews[reviews['Label'] == 1].sample(subset, random_state = 43)

# modified_data = pd.concat([pos_subset, neg_subset])
modified_data = reviews

In [7]:
modified_data = modified_data.reset_index(drop=True)
modified_data.sample(5)

,Unnamed: 0,Review,Label
3176,3176,"Much more then I expected!: When it arrived, I...",2
14113,14113,Poor NonStick Action!: We purchased this pan a...,1
663,663,"1080P ok, that is it!: I love Madonna! Just wi...",2
1134,1134,Very Pleased: Use it in my chair at work - did...,2
15066,15066,Not User-Friendly Enough for Beginners: There ...,1


In [ ]:
# modified_data.to_csv('/home/verpen/Downloads/Datasets/Amazon_Reviews_4M/train_short_version.csv')

# Data Preprocessing

In [8]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "1st" : "first", 
    "2nd" : "second", 
    "3rd" : "third",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "canx" : "cancel",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dnt" : "don't",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "emerg" : "emergency",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "haha" : "laugh", 
    "hahaha" : "laugh", 
    "hehe" : "laugh", 
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrs" : "hour",
    "hrh" : "his royal highness",    
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pls" : "please",
    "plz" : "please",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rite" : "right",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sry" : "sorry",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thank" : "thank",
    "thanks" : "thank",
    "thks" : "thank",
    "tho" : "though",
    "thx" : "thank",
    "tia" : "thanks in advance",
    "til" : "until",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "us" : "united states",
    "usa" : "united states",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "ya" : "you",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

# This abbreviation list is by @nmaguette from Kaggle

In [9]:
def expand_abbreviation(text):
    """ This function takes an abbreviated string
        and returns the expanded version with whitespace 
        replaced by underscore """
    
    expanded_abbr = ''
    expanded_text = ''
    for word in text:
        expanded_abbr = word
        if word in abbreviations: 
            expanded_abbr = abbreviations[word]
            expanded_abbr = expanded_abbr.replace(' ', '_')          
        
        expanded_text = expanded_text + expanded_abbr + " "
    
    return expanded_text

In [4]:
emot_obj = emot.core.emot()  

In [5]:
def emoj_emot_to_text(text):
    """ This function converts emojis and emoticons to corresponding text """
    
    # Emoticons  
    emoticons_list = emot_obj.emoticons(text)['mean']
    for emoticon in emoticons_list: 
        emoticon = emoticon.replace(', ', ' ')
        emoticon = emoticon.replace(' ', '_')
        text = text + emoticon + " "       
    
    
    # Emojis
    text = emoji.demojize(text) 
    text = re.sub(':', '', text)
    return text

In [6]:
emoj_emot_to_text('^_^')

'^_^Joyful '

In [12]:
stop_words = list(STOP_WORDS)

# Exclude from stop words list the following negative words
stop_words.remove('not')
stop_words.remove('never')
stop_words.remove('none')
stop_words.remove('noone')
stop_words.remove('neither')
stop_words.remove('cannot')

In [13]:
def remove_trivial_words(text):  
    """ This function Removes stopwords and words with length of one character or two """
    
    temp = ""
    for word in text.split():
        if  word not in stop_words:
            if len(word) != 1 and len(word) != 2:
                temp += word + " "
    return temp

In [14]:
# Stemming
snowballStemmer = SnowballStemmer(language='english') 
reviews_finished = 0

def preprocess_texts(text): 
    
    #     Preprocess texts
    text = text.lower() # Convert to lower case
#     text = contractions.fix(text) # Remove contractions
    
    #     Remove insignificant characters using regex
    """text = re.sub('\S+@\S+.\S+', '', text) # Remove emails
    text = re.sub('@[\S]+', '', text) # Remove mentions starting with @       
    text = re.sub('(http|https):\/\/[\S]*', '', text) # Remove links starting with http or https
    text = re.sub('#+\S*|\.+', '', text) # Remove Hashtags and dots
    text = re.sub(r'(.)\1{2,}', r'\1', text) # Remove characters repetition
    text = re.sub(r'&[\S]+;', '', text) # Remove html tags, such as &amp;  
    text = re.split('(\W+)', text) # Insert a space between characters and punctuations
       
    # Expand abbreviations
    text = expand_abbreviation(text)   
    
    # Emojis and Emoticons
    text = emoj_emot_to_text(text)      
    
    text = re.sub('\W', ' ', text)  # Remove remaining punctuations  
    text = re.sub('\d', '', text)  # Remove Numbers    
   """
    # Remove trivial words
    text = remove_trivial_words(text)
    
    # Stemming    
    stemmed_token_list = [] 
    for token in text.split(): 
        stemmed_token_list.append(snowballStemmer.stem(token))
    
    global reviews_finished
    reviews_finished += 1
    print(f'Reviews finished: {reviews_finished}')
    return stemmed_token_list #Return processed text as a list

In [15]:
modified_data['Tokens'] = modified_data['Review'].apply(preprocess_texts)

Reviews finished: 1
Reviews finished: 2
Reviews finished: 3
Reviews finished: 4
Reviews finished: 5
Reviews finished: 6
Reviews finished: 7
Reviews finished: 8
Reviews finished: 9
Reviews finished: 10
Reviews finished: 11
Reviews finished: 12
Reviews finished: 13
Reviews finished: 14
Reviews finished: 15
Reviews finished: 16
Reviews finished: 17
Reviews finished: 18
Reviews finished: 19
Reviews finished: 20
Reviews finished: 21
Reviews finished: 22
Reviews finished: 23
Reviews finished: 24
Reviews finished: 25
Reviews finished: 26
Reviews finished: 27
Reviews finished: 28
Reviews finished: 29
Reviews finished: 30
Reviews finished: 31
Reviews finished: 32
Reviews finished: 33
Reviews finished: 34
Reviews finished: 35
Reviews finished: 36
Reviews finished: 37
Reviews finished: 38
Reviews finished: 39
Reviews finished: 40
Reviews finished: 41
Reviews finished: 42
Reviews finished: 43
Reviews finished: 44
Reviews finished: 45
Reviews finished: 46
Reviews finished: 47
Reviews finished: 48
R

Reviews finished: 1176
Reviews finished: 1177
Reviews finished: 1178
Reviews finished: 1179
Reviews finished: 1180
Reviews finished: 1181
Reviews finished: 1182
Reviews finished: 1183
Reviews finished: 1184
Reviews finished: 1185
Reviews finished: 1186
Reviews finished: 1187
Reviews finished: 1188
Reviews finished: 1189
Reviews finished: 1190
Reviews finished: 1191
Reviews finished: 1192
Reviews finished: 1193
Reviews finished: 1194
Reviews finished: 1195
Reviews finished: 1196
Reviews finished: 1197
Reviews finished: 1198
Reviews finished: 1199
Reviews finished: 1200
Reviews finished: 1201
Reviews finished: 1202
Reviews finished: 1203
Reviews finished: 1204
Reviews finished: 1205
Reviews finished: 1206
Reviews finished: 1207
Reviews finished: 1208
Reviews finished: 1209
Reviews finished: 1210
Reviews finished: 1211
Reviews finished: 1212
Reviews finished: 1213
Reviews finished: 1214
Reviews finished: 1215
Reviews finished: 1216
Reviews finished: 1217
Reviews finished: 1218
Reviews fin

Reviews finished: 1651
Reviews finished: 1652
Reviews finished: 1653
Reviews finished: 1654
Reviews finished: 1655
Reviews finished: 1656
Reviews finished: 1657
Reviews finished: 1658
Reviews finished: 1659
Reviews finished: 1660
Reviews finished: 1661
Reviews finished: 1662
Reviews finished: 1663
Reviews finished: 1664
Reviews finished: 1665
Reviews finished: 1666
Reviews finished: 1667
Reviews finished: 1668
Reviews finished: 1669
Reviews finished: 1670
Reviews finished: 1671
Reviews finished: 1672
Reviews finished: 1673
Reviews finished: 1674
Reviews finished: 1675
Reviews finished: 1676
Reviews finished: 1677
Reviews finished: 1678
Reviews finished: 1679
Reviews finished: 1680
Reviews finished: 1681
Reviews finished: 1682
Reviews finished: 1683
Reviews finished: 1684
Reviews finished: 1685
Reviews finished: 1686
Reviews finished: 1687
Reviews finished: 1688
Reviews finished: 1689
Reviews finished: 1690
Reviews finished: 1691
Reviews finished: 1692
Reviews finished: 1693
Reviews fin

Reviews finished: 2106
Reviews finished: 2107
Reviews finished: 2108
Reviews finished: 2109
Reviews finished: 2110
Reviews finished: 2111
Reviews finished: 2112
Reviews finished: 2113
Reviews finished: 2114
Reviews finished: 2115
Reviews finished: 2116
Reviews finished: 2117
Reviews finished: 2118
Reviews finished: 2119
Reviews finished: 2120
Reviews finished: 2121
Reviews finished: 2122
Reviews finished: 2123
Reviews finished: 2124
Reviews finished: 2125
Reviews finished: 2126
Reviews finished: 2127
Reviews finished: 2128
Reviews finished: 2129
Reviews finished: 2130
Reviews finished: 2131
Reviews finished: 2132
Reviews finished: 2133
Reviews finished: 2134
Reviews finished: 2135
Reviews finished: 2136
Reviews finished: 2137
Reviews finished: 2138
Reviews finished: 2139
Reviews finished: 2140
Reviews finished: 2141
Reviews finished: 2142
Reviews finished: 2143
Reviews finished: 2144
Reviews finished: 2145
Reviews finished: 2146
Reviews finished: 2147
Reviews finished: 2148
Reviews fin

Reviews finished: 3429
Reviews finished: 3430
Reviews finished: 3431
Reviews finished: 3432
Reviews finished: 3433
Reviews finished: 3434
Reviews finished: 3435
Reviews finished: 3436
Reviews finished: 3437
Reviews finished: 3438
Reviews finished: 3439
Reviews finished: 3440
Reviews finished: 3441
Reviews finished: 3442
Reviews finished: 3443
Reviews finished: 3444
Reviews finished: 3445
Reviews finished: 3446
Reviews finished: 3447
Reviews finished: 3448
Reviews finished: 3449
Reviews finished: 3450
Reviews finished: 3451
Reviews finished: 3452
Reviews finished: 3453
Reviews finished: 3454
Reviews finished: 3455
Reviews finished: 3456
Reviews finished: 3457
Reviews finished: 3458
Reviews finished: 3459
Reviews finished: 3460
Reviews finished: 3461
Reviews finished: 3462
Reviews finished: 3463
Reviews finished: 3464
Reviews finished: 3465
Reviews finished: 3466
Reviews finished: 3467
Reviews finished: 3468
Reviews finished: 3469
Reviews finished: 3470
Reviews finished: 3471
Reviews fin

...

Reviews finished: 13238
Reviews finished: 13239
Reviews finished: 13240
Reviews finished: 13241
Reviews finished: 13242
Reviews finished: 13243
Reviews finished: 13244
Reviews finished: 13245
Reviews finished: 13246
Reviews finished: 13247
Reviews finished: 13248
Reviews finished: 13249
Reviews finished: 13250
Reviews finished: 13251
Reviews finished: 13252
Reviews finished: 13253
Reviews finished: 13254
Reviews finished: 13255
Reviews finished: 13256
Reviews finished: 13257
Reviews finished: 13258
Reviews finished: 13259
Reviews finished: 13260
Reviews finished: 13261
Reviews finished: 13262
Reviews finished: 13263
Reviews finished: 13264
Reviews finished: 13265
Reviews finished: 13266
Reviews finished: 13267
Reviews finished: 13268
Reviews finished: 13269
Reviews finished: 13270
Reviews finished: 13271
Reviews finished: 13272
Reviews finished: 13273
Reviews finished: 13274
Reviews finished: 13275
Reviews finished: 13276
Reviews finished: 13277
Reviews finished: 13278
Reviews finished

Reviews finished: 14738
Reviews finished: 14739
Reviews finished: 14740
Reviews finished: 14741
Reviews finished: 14742
Reviews finished: 14743
Reviews finished: 14744
Reviews finished: 14745
Reviews finished: 14746
Reviews finished: 14747
Reviews finished: 14748
Reviews finished: 14749
Reviews finished: 14750
Reviews finished: 14751
Reviews finished: 14752
Reviews finished: 14753
Reviews finished: 14754
Reviews finished: 14755
Reviews finished: 14756
Reviews finished: 14757
Reviews finished: 14758
Reviews finished: 14759
Reviews finished: 14760
Reviews finished: 14761
Reviews finished: 14762
Reviews finished: 14763
Reviews finished: 14764
Reviews finished: 14765
Reviews finished: 14766
Reviews finished: 14767
Reviews finished: 14768
Reviews finished: 14769
Reviews finished: 14770
Reviews finished: 14771
Reviews finished: 14772
Reviews finished: 14773
Reviews finished: 14774
Reviews finished: 14775
Reviews finished: 14776
Reviews finished: 14777
Reviews finished: 14778
Reviews finished

Reviews finished: 16737
Reviews finished: 16738
Reviews finished: 16739
Reviews finished: 16740
Reviews finished: 16741
Reviews finished: 16742
Reviews finished: 16743
Reviews finished: 16744
Reviews finished: 16745
Reviews finished: 16746
Reviews finished: 16747
Reviews finished: 16748
Reviews finished: 16749
Reviews finished: 16750
Reviews finished: 16751
Reviews finished: 16752
Reviews finished: 16753
Reviews finished: 16754
Reviews finished: 16755
Reviews finished: 16756
Reviews finished: 16757
Reviews finished: 16758
Reviews finished: 16759
Reviews finished: 16760
Reviews finished: 16761
Reviews finished: 16762
Reviews finished: 16763
Reviews finished: 16764
Reviews finished: 16765
Reviews finished: 16766
Reviews finished: 16767
Reviews finished: 16768
Reviews finished: 16769
Reviews finished: 16770
Reviews finished: 16771
Reviews finished: 16772
Reviews finished: 16773
Reviews finished: 16774
Reviews finished: 16775
Reviews finished: 16776
Reviews finished: 16777
Reviews finished

Reviews finished: 17737
Reviews finished: 17738
Reviews finished: 17739
Reviews finished: 17740
Reviews finished: 17741
Reviews finished: 17742
Reviews finished: 17743
Reviews finished: 17744
Reviews finished: 17745
Reviews finished: 17746
Reviews finished: 17747
Reviews finished: 17748
Reviews finished: 17749
Reviews finished: 17750
Reviews finished: 17751
Reviews finished: 17752
Reviews finished: 17753
Reviews finished: 17754
Reviews finished: 17755
Reviews finished: 17756
Reviews finished: 17757
Reviews finished: 17758
Reviews finished: 17759
Reviews finished: 17760
Reviews finished: 17761
Reviews finished: 17762
Reviews finished: 17763
Reviews finished: 17764
Reviews finished: 17765
Reviews finished: 17766
Reviews finished: 17767
Reviews finished: 17768
Reviews finished: 17769
Reviews finished: 17770
Reviews finished: 17771
Reviews finished: 17772
Reviews finished: 17773
Reviews finished: 17774
Reviews finished: 17775
Reviews finished: 17776
Reviews finished: 17777
Reviews finished

Reviews finished: 18736
Reviews finished: 18737
Reviews finished: 18738
Reviews finished: 18739
Reviews finished: 18740
Reviews finished: 18741
Reviews finished: 18742
Reviews finished: 18743
Reviews finished: 18744
Reviews finished: 18745
Reviews finished: 18746
Reviews finished: 18747
Reviews finished: 18748
Reviews finished: 18749
Reviews finished: 18750
Reviews finished: 18751
Reviews finished: 18752
Reviews finished: 18753
Reviews finished: 18754
Reviews finished: 18755
Reviews finished: 18756
Reviews finished: 18757
Reviews finished: 18758
Reviews finished: 18759
Reviews finished: 18760
Reviews finished: 18761
Reviews finished: 18762
Reviews finished: 18763
Reviews finished: 18764
Reviews finished: 18765
Reviews finished: 18766
Reviews finished: 18767
Reviews finished: 18768
Reviews finished: 18769
Reviews finished: 18770
Reviews finished: 18771
Reviews finished: 18772
Reviews finished: 18773
Reviews finished: 18774
Reviews finished: 18775
Reviews finished: 18776
Reviews finished

In [16]:
modified_data

,Unnamed: 0,Review,Label,Tokens
0,0,Good novel about gay youth: There are two book...,2,"[good, novel, gay, youth:, book, gay, romanc, ..."
1,1,"Tudors Season I: This series is ""Awesome!"" It ...",2,"[tudor, season, seri, ""awesome!"", addicting., ..."
2,2,Ah the memories.: What can be said about Rocko...,2,"[memories.:, said, rocko, modern, life?, bizar..."
3,3,Cancer is not a Disese by Andreas Moritz: A ve...,2,"[cancer, not, dises, andrea, moritz:, refresh,..."
4,4,better late than never: I more or less stumble...,2,"[better, late, never:, stumbl, novel, fact, en..."
...,...,...,...,...
19995,19995,2-Inch Visco Elastic Memory Foam Mattress Topp...,1,"[2-inch, visco, elast, memori, foam, mattress,..."
19996,19996,This doesn't deserve ONE star!: If you like al...,1,"[doesn't, deserv, star!:, like, band, play, tr..."
19997,19997,TERRIBLE!!!!: This story is HORRIBLE!!! It jus...,1,"[terrible!!!!:, stori, horrible!!!, drag, on,,..."
19998,19998,"Half a star actually, if that..: As an artist ...",1,"[half, star, actually,, that..:, artist, awar,..."


In [17]:
# Convert token lists to string 
def convert_list_str(tokens):
    return ' '.join(tokens)
modified_data['String_tokens'] = modified_data['Tokens'].apply(convert_list_str)
modified_data = modified_data.reset_index(drop=True)
modified_data

,Unnamed: 0,Review,Label,Tokens,String_tokens
0,0,Good novel about gay youth: There are two book...,2,"[good, novel, gay, youth:, book, gay, romanc, ...",good novel gay youth: book gay romanc fiction ...
1,1,"Tudors Season I: This series is ""Awesome!"" It ...",2,"[tudor, season, seri, ""awesome!"", addicting., ...","tudor season seri ""awesome!"" addicting. start,..."
2,2,Ah the memories.: What can be said about Rocko...,2,"[memories.:, said, rocko, modern, life?, bizar...",memories.: said rocko modern life? bizarr look...
3,3,Cancer is not a Disese by Andreas Moritz: A ve...,2,"[cancer, not, dises, andrea, moritz:, refresh,...",cancer not dises andrea moritz: refresh new lo...
4,4,better late than never: I more or less stumble...,2,"[better, late, never:, stumbl, novel, fact, en...",better late never: stumbl novel fact enjoy fan...
...,...,...,...,...,...
19995,19995,2-Inch Visco Elastic Memory Foam Mattress Topp...,1,"[2-inch, visco, elast, memori, foam, mattress,...",2-inch visco elast memori foam mattress topper...
19996,19996,This doesn't deserve ONE star!: If you like al...,1,"[doesn't, deserv, star!:, like, band, play, tr...","doesn't deserv star!: like band play tribute, ..."
19997,19997,TERRIBLE!!!!: This story is HORRIBLE!!! It jus...,1,"[terrible!!!!:, stori, horrible!!!, drag, on,,...","terrible!!!!: stori horrible!!! drag on, chang..."
19998,19998,"Half a star actually, if that..: As an artist ...",1,"[half, star, actually,, that..:, artist, awar,...","half star actually, that..: artist awar diffic..."


# Analyze data after processing

In [ ]:
# Count the number of tokens in the dataset after processing
num_unique_tokens = 0
num_of_total_tokens = 0
num_pos_tokens = 0
num_neg_tokens = 0
pos_tokens = []
neg_tokens = []
rows = modified_data.shape[0]

for i in range(rows):
    if modified_data['Label'][i] == 1:
        for token in modified_data['Tokens'][i]:
            neg_tokens.append(token)    
    else:
        for token in modified_data['Tokens'][i]:
            pos_tokens.append(token)     

num_pos_tokens = len(pos_tokens)
num_neg_tokens = len(neg_tokens)
num_of_total_tokens = num_pos_tokens + num_neg_tokens
num_unique_tokens = len(set(pos_tokens)) + len(set(neg_tokens))

print(f"""Number of tokens: {num_of_total_tokens}
Number of unique tokens: {num_unique_tokens}
Positive Tokens: {num_pos_tokens}
Negative Tokens: {num_neg_tokens}
Positive Unique Tokens: {len(set(pos_tokens))}
Negative Unique Tokens: {len(set(neg_tokens))}""")

In [ ]:
# Calculate words occurances 
def occurances_of_words(data, reverse=True):
    freq_words_dic = {}
    rows = data.shape[0]
    
    for i in range(rows):
        for word in data['Tokens'][i]:
            if word in freq_words_dic:
                freq = freq_words_dic[word]
                freq_words_dic[word] = freq + 1
            else:
                freq_words_dic[word] = 1
    return sorted(freq_words_dic.items(), key=lambda x: x[1], reverse=reverse)

In [ ]:
# Calculate words occurances in negative reviews
neg_modified_data = modified_data[modified_data['Label'] == 1]
neg_modified_data = neg_modified_data.reset_index(drop=True)

neg_freq_words = occurances_of_words(neg_modified_data)
sorted(neg_freq_words, key=lambda x: x[1], reverse=True)[:20]

In [ ]:
# Calculate words occurances in positive reviews
pos_modified_data = modified_data[modified_data['Label'] == 2]
pos_modified_data = pos_modified_data.reset_index(drop=True)

pos_freq_words = occurances_of_words(pos_modified_data)
sorted(pos_freq_words, key=lambda x: x[1], reverse=True)[:20]

In [ ]:
def plot_freq_words(freq_words_list, label, color):
    """ Plot a bar chart with the most frequent words """

    freq_words = [f_item[0] for f_item in freq_words_list]
    occurances = [s_item[1] for s_item in freq_words_list]    
    
    # X ticks position
    x_pos = [x*2 for x in range(20)]
    
    plt.figure(figsize=(12, 7))
    plt.bar(x_pos, occurances, align='center', width=1.5, alpha=0.8, color=color, label=label)
    plt.xlabel('Most Frequent Words')
    plt.ylabel('Number of occurances')
    plt.legend()
    plt.xticks(x_pos, freq_words, rotation='vertical')
    plt.show()

In [ ]:
# Plot a bar chart for neg words
plot_freq_words(neg_freq_words[:20], 'Negative Words', 'red')

# Plot a bar chart for pos words
plot_freq_words(pos_freq_words[:20], 'Positive Words', 'green')

In [ ]:
# Display a word cloud
def plot_wordcloud(text, words):

    wc = WordCloud(width = 1920, height = 1080, background_color = 'black', max_words = words, colormap='Set2')
    wc.generate(text)    
    
    plt.figure(figsize=(10,10), facecolor="black")
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.imshow(wc, interpolation='bilinear')
    plt.show()

In [ ]:
# Extract frequent words 
wc_neg_text = [word[0] for word in neg_freq_words]
wc_neg_text = ' '.join(wc_neg_text)

wc_pos_text = [word[0] for word in pos_freq_words]
wc_pos_text = ' '.join(wc_pos_text)

In [ ]:
max_words = 150
print('Negative Frequent Words: ')
plot_wordcloud(wc_neg_text, max_words)

print('\nPositive Frequent Words: ')
plot_wordcloud(wc_pos_text, max_words)

In [ ]:
# Deep learning - RNN
X = modified_data['String_tokens'].values
y = modified_data['Label']

In [ ]:
print(f'X: {X.shape}')
print(f'y: {y.shape}')

In [ ]:
# from keras.preprocessing.text import Tokenizer
# from keras_preprocessing.sequence import pad_sequences

# tk = Tokenizer(lower = True)
# tk.fit_on_texts(X)
# X_seq = tk.texts_to_sequences(X)
# X_pad = pad_sequences(X_seq, maxlen=100, padding='post')

In [ ]:
# X_pad 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size = 0.25, random_state = 1)

In [ ]:
batch_size = 64
X_train1 = X_train[batch_size:]
y_train1 = y_train[batch_size:]
X_valid = X_train[:batch_size]
y_valid = y_train[:batch_size]

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
vocabulary_size = len(tk.word_counts.keys())+1
max_words = 100
embedding_size = 32
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(200))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
results = model.fit(X_train1, y_train1, batch_size=1024, epochs=5, validation_data=(X_valid, y_valid))

#  Test Data


In [18]:
# Test data 
reviews_test = pd.read_csv("/home/verpen/Downloads/Datasets/Amazon_Reviews_4M/test.csv")

In [21]:
# Choose a random subset from the dataset with equal number of 
# positive and negative tweets
subset = 1000
pos_subset = reviews_test[reviews_test['Label'] == 2].sample(subset, random_state = 43)
neg_subset = reviews_test[reviews_test['Label'] == 1].sample(subset, random_state = 43)

modified_test_data = pd.concat([pos_subset, neg_subset])
# modified_data = reviews

In [22]:
modified_test_data

,Review,Label
299637,"Funny, imaginative: The Wide-mouthed Frog is a...",2
197870,Wonderful!: This book should not have been nec...,2
208895,Mario Kart DS: This is a must buy if your kids...,2
189862,My (macho) Husband read it first and loved it!...,2
155692,Worked: Rewarded my daughter with stickers and...,2
...,...,...
194610,"Dont buy: I bought, and returned yesterday, th...",1
301864,Worthless: Not quite a beautiful picture book....,1
170823,Useless: I don't understand this product. The ...,1
215080,"No Subtitles: Beautiful movie, but in German. ...",1


In [23]:
modified_test_data['Tokens'] = modified_test_data['Review'].apply(preprocess_texts)
modified_test_data['String_tokens'] = modified_test_data['Tokens'].apply(convert_list_str)
modified_test_data

Reviews finished: 20001
Reviews finished: 20002
Reviews finished: 20003
Reviews finished: 20004
Reviews finished: 20005
Reviews finished: 20006
Reviews finished: 20007
Reviews finished: 20008
Reviews finished: 20009
Reviews finished: 20010
Reviews finished: 20011
Reviews finished: 20012
Reviews finished: 20013
Reviews finished: 20014
Reviews finished: 20015
Reviews finished: 20016
Reviews finished: 20017
Reviews finished: 20018
Reviews finished: 20019
Reviews finished: 20020
Reviews finished: 20021
Reviews finished: 20022
Reviews finished: 20023
Reviews finished: 20024
Reviews finished: 20025
Reviews finished: 20026
Reviews finished: 20027
Reviews finished: 20028
Reviews finished: 20029
Reviews finished: 20030
Reviews finished: 20031
Reviews finished: 20032
Reviews finished: 20033
Reviews finished: 20034
Reviews finished: 20035
Reviews finished: 20036
Reviews finished: 20037
Reviews finished: 20038
Reviews finished: 20039
Reviews finished: 20040
Reviews finished: 20041
Reviews finished

Reviews finished: 20790
Reviews finished: 20791
Reviews finished: 20792
Reviews finished: 20793
Reviews finished: 20794
Reviews finished: 20795
Reviews finished: 20796
Reviews finished: 20797
Reviews finished: 20798
Reviews finished: 20799
Reviews finished: 20800
Reviews finished: 20801
Reviews finished: 20802
Reviews finished: 20803
Reviews finished: 20804
Reviews finished: 20805
Reviews finished: 20806
Reviews finished: 20807
Reviews finished: 20808
Reviews finished: 20809
Reviews finished: 20810
Reviews finished: 20811
Reviews finished: 20812
Reviews finished: 20813
Reviews finished: 20814
Reviews finished: 20815
Reviews finished: 20816
Reviews finished: 20817
Reviews finished: 20818
Reviews finished: 20819
Reviews finished: 20820
Reviews finished: 20821
Reviews finished: 20822
Reviews finished: 20823
Reviews finished: 20824
Reviews finished: 20825
Reviews finished: 20826
Reviews finished: 20827
Reviews finished: 20828
Reviews finished: 20829
Reviews finished: 20830
Reviews finished

Reviews finished: 21415
Reviews finished: 21416
Reviews finished: 21417
Reviews finished: 21418
Reviews finished: 21419
Reviews finished: 21420
Reviews finished: 21421
Reviews finished: 21422
Reviews finished: 21423
Reviews finished: 21424
Reviews finished: 21425
Reviews finished: 21426
Reviews finished: 21427
Reviews finished: 21428
Reviews finished: 21429
Reviews finished: 21430
Reviews finished: 21431
Reviews finished: 21432
Reviews finished: 21433
Reviews finished: 21434
Reviews finished: 21435
Reviews finished: 21436
Reviews finished: 21437
Reviews finished: 21438
Reviews finished: 21439
Reviews finished: 21440
Reviews finished: 21441
Reviews finished: 21442
Reviews finished: 21443
Reviews finished: 21444
Reviews finished: 21445
Reviews finished: 21446
Reviews finished: 21447
Reviews finished: 21448
Reviews finished: 21449
Reviews finished: 21450
Reviews finished: 21451
Reviews finished: 21452
Reviews finished: 21453
Reviews finished: 21454
Reviews finished: 21455
Reviews finished

Reviews finished: 21921
Reviews finished: 21922
Reviews finished: 21923
Reviews finished: 21924
Reviews finished: 21925
Reviews finished: 21926
Reviews finished: 21927
Reviews finished: 21928
Reviews finished: 21929
Reviews finished: 21930
Reviews finished: 21931
Reviews finished: 21932
Reviews finished: 21933
Reviews finished: 21934
Reviews finished: 21935
Reviews finished: 21936
Reviews finished: 21937
Reviews finished: 21938
Reviews finished: 21939
Reviews finished: 21940
Reviews finished: 21941
Reviews finished: 21942
Reviews finished: 21943
Reviews finished: 21944
Reviews finished: 21945
Reviews finished: 21946
Reviews finished: 21947
Reviews finished: 21948
Reviews finished: 21949
Reviews finished: 21950
Reviews finished: 21951
Reviews finished: 21952
Reviews finished: 21953
Reviews finished: 21954
Reviews finished: 21955
Reviews finished: 21956
Reviews finished: 21957
Reviews finished: 21958
Reviews finished: 21959
Reviews finished: 21960
Reviews finished: 21961
Reviews finished

,Review,Label,Tokens,String_tokens
299637,"Funny, imaginative: The Wide-mouthed Frog is a...",2,"[funny,, imaginative:, wide-mouth, frog, wonde...","funny, imaginative: wide-mouth frog wonder ill..."
197870,Wonderful!: This book should not have been nec...,2,"[wonderful!:, book, not, necessary., effort, s...",wonderful!: book not necessary. effort scholar...
208895,Mario Kart DS: This is a must buy if your kids...,2,"[mario, kart, ds:, buy, kid, like, race, game,...",mario kart ds: buy kid like race game like dau...
189862,My (macho) Husband read it first and loved it!...,2,"[(macho), husband, read, love, it!!:, southern...",(macho) husband read love it!!: southern rais ...
155692,Worked: Rewarded my daughter with stickers and...,2,"[worked:, reward, daughter, sticker, love, act...",worked: reward daughter sticker love actual bo...
...,...,...,...,...
194610,"Dont buy: I bought, and returned yesterday, th...",1,"[dont, buy:, bought,, return, yesterday,, prod...","dont buy: bought, return yesterday, product. t..."
301864,Worthless: Not quite a beautiful picture book....,1,"[worthless:, not, beauti, pictur, book., certa...",worthless: not beauti pictur book. certain not...
170823,Useless: I don't understand this product. The ...,1,"[useless:, don't, understand, product., clip, ...",useless: don't understand product. clip not wo...
215080,"No Subtitles: Beautiful movie, but in German. ...",1,"[subtitles:, beauti, movie,, german., english,...","subtitles: beauti movie, german. english subti..."


In [24]:
vocab_size = 10000
embedding_dim = 100
max_length = 300
trunc_type='post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(modified_data['String_tokens'])
# tokenizer.fit_on_texts(modified_test_data['String_tokens'])
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(modified_data['String_tokens'])
X_train = pad_sequences(train_sequences,maxlen=max_length, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(modified_test_data['String_tokens'])
X_test = pad_sequences(test_sequences,maxlen=max_length, truncating=trunc_type)

In [25]:
vocab_size = 10000
embedding_dim = 100
max_length = 300

In [26]:
X_train

array([[   0,    0,    0, ...,   29, 1641,  478],
       [   0,    0,    0, ...,   11,    4, 1442],
       [   0,    0,    0, ..., 2851,  753, 5625],
       ...,
       [   0,    0,    0, ...,    4,  130, 2184],
       [   0,    0,    0, ...,  112,   21,   94],
       [   0,    0,    0, ...,    1,    1, 1457]], dtype=int32)

In [27]:
model=Sequential()
model.add(Embedding(vocab_size,embedding_dim,input_length=max_length))
model.add(BatchNormalization())
model.add(LSTM(10))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=['accuracy'])

2022-10-19 10:47:12.220096: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [28]:
h = model.fit(X_train, modified_data['Label'], batch_size=1024, epochs=5, validation_data=(X_test, modified_test_data['Label']))

Epoch 1/5
20/20 [==============================] - 19s 796ms/step - loss: -0.4580 - accuracy: 0.4895 - val_loss: 0.3934 - val_accuracy: 0.5000
Epoch 2/5
20/20 [==============================] - 15s 772ms/step - loss: -1.3970 - accuracy: 0.5000 - val_loss: 0.1826 - val_accuracy: 0.5000
Epoch 3/5
20/20 [==============================] - 16s 800ms/step - loss: -1.6355 - accuracy: 0.5000 - val_loss: -0.0023 - val_accuracy: 0.5000
Epoch 4/5
20/20 [==============================] - 16s 787ms/step - loss: -1.7848 - accuracy: 0.5000 - val_loss: -0.1851 - val_accuracy: 0.5000
Epoch 5/5
20/20 [==============================] - 16s 808ms/step - loss: -1.9143 - accuracy: 0.5000 - val_loss: -0.3723 - val_accuracy: 0.5000


# Feature Extraction and Model Training

# Saving the best model

In [ ]:
pickle.dump(log_model, open('/home/verpen/Downloads/Datasets/Models/SA_Amazon_Reviews_LogReg_91.pkl', 'wb'))
pickle.dusvm_model_linearsvm_model_linear, open('/home/verpen/Downloads/Datasets/Models/SA_Amazon_Reviews_SVM_91.pkl', 'wb'))

# Testing the model 

In [ ]:
def pipeline(model, test_data):
    
    # Preprocessing
    print("Processing texts....")
    start = time.perf_counter()
    test_data['processed_texts'] = test_data['Review'].apply(preprocess_texts)
    test_data['string_texts'] = test_data['processed_texts'].apply(convert_list_str)
    end = time.perf_counter()    
    print(f'Processing time: {end - start}')
    
    # Transforming processed text using tfidf vectorizer initiated and fitted above
    print("Transforming texts to features....")
    test_tfidf = tfidf.transform(test_data['string_texts'])
    
    # Predict
    print("Predicting....")
    predictions = model.predict(test_tfidf)
    
    # Print a Classification report
    print(classification_report(test_data['Label'], predictions))
    cm = confusion_matrix(test_data['Label'], predictions, labels=[1, 2])
    print(cm)
    
    # Plot a heatmap
    plt.figure(figsize= (10, 7))
    sn.heatmap(cm, annot=True, fmt='.0f')
    plt.xlabel('Predicted')
    plt.ylabel('value')
    plt.show()
    
    # Plot a pie 
    (unique, counts) = np.unique(predictions, return_counts=True)
    print(f'Unique: {unique}\t\tCounts: {counts}')
    
    neg_texts_counts = counts[0]
    pos_texts_counts = counts[1]
    labels = ["Negative", "Positive"]
    plt.pie(counts, labels=labels, shadow=True, autopct="%1.2f%%")

# Model testing with Amazon Reviews dataset

In [ ]:
# Another test
amazon_reviews = pd.read_csv('/home/verpen/Downloads/Datasets/Amazon_Reviews_4M/test.csv') # 400k reviews
amazon_reviews = amazon_reviews.sample(100000)
amazon_reviews = amazon_reviews.reset_index(drop=True)
amazon_reviews

In [ ]:
pipeline(svm_model_linear, amazon_reviews)

# Model testing with IMDb Movie Reviews dataset

In [ ]:
movie_reviews = pd.read_csv('/home/verpen/Downloads/Datasets/imdb/IMDb_Moview_Reviews.zip')
movie_reviews = movie_reviews.rename(columns={'review':'Review', 'sentiment': 'Label'})
movie_reviews

In [ ]:
movie_reviews['Label'] = movie_reviews['Label'].apply(lambda x: 2 if x == 'positive' else 1)
movie_reviews

In [ ]:
pipeline(svm_model_linear, movie_reviews)

## Model testing with Pang's Movie Reviews dataset

In [ ]:
# https://github.com/anurag3290/IMDB-Sentiment-Analysis
pang_movies_1 = pd.read_csv('/home/verpen/Downloads/Datasets/Pang/Dataset/IMDB-Sentiment-Analysis-master/movie_review_train.csv')
pang_movies_2 = pd.read_csv('/home/verpen/Downloads/Datasets/Pang/Dataset/IMDB-Sentiment-Analysis-master/movie_review_test.csv')
pang_movies = pd.concat([pang_movies_1, pang_movies_2])
pang_movies

In [ ]:
pang_movies["class"] = pang_movies['class'].apply(lambda x: 2 if x == 'Pos' else 1)
pang_movies

In [ ]:
pang_movies = pang_movies.rename(columns={'text': 'Review', 'class': 'Label'})
pang_movies

In [ ]:
pipeline(svm_model_linear, pang_movies)

In [ ]:
modified_data

In [ ]:
#convert each line text into json and form the json array
reviewsjson = []
for review in modified_data['Review']:
    print(review)
#     if(review != ''):
#         reviewsjson.append(json.loads(review))

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib
from string import punctuation
import json
from collections import Counter

In [ ]:
#a function for replacing characters and converting special form of words
def replaceSpecialChars(reviewtext):
    reviewtext = reviewtext.replace('.', ' ')
    reviewtext = reviewtext.replace(',', ' ')
    reviewtext = reviewtext.replace(';', ' ')
    reviewtext = reviewtext.replace(':', ' ')
    reviewtext = reviewtext.replace(',', ' ')
    reviewtext = reviewtext.replace('?', ' ')
    reviewtext = reviewtext.replace('!', ' ')
    reviewtext = reviewtext.replace('*', ' ')
    reviewtext = reviewtext.replace('&', ' ')
    reviewtext = reviewtext.replace('(', ' ')
    reviewtext = reviewtext.replace(')', ' ')
    reviewtext = reviewtext.replace('[', ' ')
    reviewtext = reviewtext.replace(']', ' ')
    reviewtext = reviewtext.replace('{', ' ')
    reviewtext = reviewtext.replace('}', ' ')
    reviewtext = reviewtext.replace('/', ' ')
    reviewtext = reviewtext.replace('\\', ' ')
    reviewtext = reviewtext.replace('-', ' ')
    reviewtext = reviewtext.replace('+', ' ')
    reviewtext = reviewtext.replace('\'s', ' ')
    return reviewtext

#prepare all text vocabulary and word2int dictionary, etc
allwords = set() # this is the word universe
allwords_counter = Counter() # this will be used to clip most frequently occurring and least frequently occurring words
positivewords_counter = Counter() #to identify words that occur more frequently in positive reviews
negativewords_counter = Counter() #to identify words that occur more frequently in negative reviews
for review in reviewsjson:
    reviewtext = review['reviewText'].lower()
    reviewtext = replaceSpecialChars(reviewtext)
    words = reviewtext.split(' ')
    allwords.update(words)
    allwords_counter.update(words)
    if int(review['overall'])>=3: #consider positive
        positivewords_counter.update(words)
    else:
        negativewords_counter.update(words)
print('finished preprocessing the review texts.')